# Project 1: Standard ECC Key Exchange

This notebook demonstrates the logic and implementation of the **Elliptic Curve Diffie-Hellman (ECDH)** key exchange.

We will show this in three parts:
1.  **The Conceptual Analogy:** The "Secret Paint Mixing"
2.  **The Mathematical Logic:** A "Toy Example" with small numbers.
3.  **The Real Implementation:** A "Black Box" demo using a secure library.

The goal is to prove how two people (Alice and Bob) can agree on a shared secret, even while an eavesdropper (Eve) is listening.

## Part 1: The "Secret Paint Mixing" Simulation (The Concept)

This shows the *logic* of how two people create a shared secret.

### 1. The Setup (The Public Room)
Everyone in the room (Alice, Bob, and Eve the Eavesdropper) agrees on a public, starting paint color.
* **Public Paint: 🟡 YELLOW**

### 2. The Secret Keys (Not Shared)
Alice and Bob each secretly pick their own private color. *No one* else knows these.
* **Alice's Secret:** 🔴 RED
* **Bob's Secret:** 🔵 BLUE

### 3. The Public Exchange (What Eve Sees)
They mix their secret color with the public color and share the result.
* Alice mixes: 🟡 YELLOW + 🔴 RED = **🟠 ORANGE**. She shouts "ORANGE!"
* Bob mixes: 🟡 YELLOW + 🔵 BLUE = **🟢 GREEN**. He shouts "GREEN!"

### 4. Eve's Problem
Eve has been listening! She knows:
1.  The public starting paint: 🟡 YELLOW
2.  Alice's public mix: 🟠 ORANGE
3.  Bob's public mix: 🟢 GREEN

But she can't "un-mix" 🟠 ORANGE to find 🔴 RED. This "un-mixing" problem is the core of ECC's security. **Eve is STUCK.**

### 5. The "Aha!" Moment (The Final Secret)
Now, Alice and Bob take what they received and add their *own* secret color.
* **Alice computes:** 🟢 GREEN (from Bob) + 🔴 RED (her secret) = **🟤 BROWN**
* **Bob computes:** 🟠 ORANGE (from Alice) + 🔵 BLUE (his secret) = **- BROWN**

### 6. The Result
**SUCCESS!** Alice and Bob have both, independently, created the *exact same* secret (🟤 BROWN) without ever sending it. Eve is left with 🟠 ORANGE and 🟢 GREEN, and cannot find their secret.

## Part 2: The "Toy Example" Math (The Logic)

This shows the same logic as the paint mixing, but using small, verifiable numbers to represent the math.

### 1. The Public Setup (The "Yellow Paint" 🟡)
Everyone (Alice, Bob, and Eve) agrees on these public parameters:
* **Prime (p):** `p = 17` (Means all math "wraps around" at 17)
* **Curve:** `y^2 = x^3 + 2x + 2 (mod 17)`
* **Generator (G):** `G = (5, 1)` (This is our public starting point)

### 2. The Secret Keys (The "Secret Colors" 🔴🔵)
Alice and Bob secretly choose a number. **These are never shared.**
* **Alice's Secret (a):** `a = 6`
* **Bob's Secret (b):** `b = 10`

### 3. The Public Exchange (The "Public Mixes" 🟠🟢)
They "multiply" their secret number by the public point `G`. This is the "hard" math (Scalar Multiplication).
* **Alice computes:** `A = a * G = 6 * (5, 1)`. The result (after complex math) is **`A = (16, 13)`**. She sends this point to Bob.
* **Bob computes:** `B = b * G = 10 * (5, 1)`. The result (after complex math) is **`B = (7, 11)`**. He sends this point to Alice.

### 4. Eve's Problem
Eve sees `G=(5,1)`, `A=(16,13)`, and `B=(7,11)`. Her problem is to find `a` from the equation `(16,13) = a * (5,1)`. This is the "un-mixing" problem (the Elliptic Curve Discrete Logarithm Problem). **Eve is STUCK.**

### 5. The "Aha!" Moment (The Final Secret 🟤)
They use their *secret* number on the *public* point they received.
* **Alice computes:** `S_A = a * B = 6 * (7, 11)`.
    * *Logically, this is:* `S_A = 6 * (10 * G) = (6 * 10) * G = 60 * G`
* **Bob computes:** `S_B = b * A = 10 * (16, 13)`.
    * *Logically, this is:* `S_B = 10 * (6 * G) = (10 * 6) * G = 60 * G`

### 6. The Result
**SUCCESS!** Since `(6 * 10) = (10 * 6)`, their final secret calculation **must be identical**. They have both arrived at the same secret point (`60 * G`), which they can now use to create a shared key for AES.

## Part 3: The 'Real Library' (Black Box) Demo

This final section uses a real, secure crypto library (`python-cryptography`) to perform the **exact same** key exchange logic we just demonstrated.

The math is identical to the "toy" example, but the numbers are 384-bits long (massive!) and the curve is a standard, secure one.

In [1]:
!pip3 install cryptography

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 5.2 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [cryptography] [cryptography]


In [2]:
# We'll use the "Elliptic Curve" (ec) tools from the library
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes
import time 

print("(The 'Public Room')")
# Everyone agrees on a public "curve"
# We'll use a standard, secure one called SECP384R1.
print("  - Public Curve: SECP384R1")
print("    (This is our '🟡 YELLOW' paint, but at secure scale)\n")
curve = ec.SECP384R1()
time.sleep(1)

(The 'Public Room')
  - Public Curve: SECP384R1
    (This is our '🟡 YELLOW' paint, but at secure scale)



In [3]:
print("--- 2. ALICE'S SIDE ---")
# Alice generates her PRIVATE key (a giant secret number 'a').
alice_private_key = ec.generate_private_key(curve)
print("  - Alice creates her private key (SECRET 'a' / '🔴 RED')")

# Alice generates her PUBLIC key (the point A = a * G).
alice_public_key = alice_private_key.public_key()
print("  - Alice creates her public key (PUBLIC 'A' / '🟠 ORANGE')")
print("    (She would send this to Bob)\n")
time.sleep(1)

--- 2. ALICE'S SIDE ---
  - Alice creates her private key (SECRET 'a' / '🔴 RED')
  - Alice creates her public key (PUBLIC 'A' / '🟠 ORANGE')
    (She would send this to Bob)



In [4]:
print("--- 3. BOB'S SIDE ---")
# Bob generates his PRIVATE key (a different giant secret number 'b').
bob_private_key = ec.generate_private_key(curve)
print("  - Bob creates his private key (SECRET 'b' / '🔵 BLUE')")

# Bob generates his PUBLIC key (the point B = b * G).
bob_public_key = bob_private_key.public_key()
print("  - Bob creates his public key (PUBLIC 'B' / '🟢 GREEN')")
print("    (He would send this to Alice)\n")
time.sleep(1)

--- 3. BOB'S SIDE ---
  - Bob creates his private key (SECRET 'b' / '🔵 BLUE')
  - Bob creates his public key (PUBLIC 'B' / '🟢 GREEN')
    (He would send this to Alice)



In [6]:
print("(The Real Exchange)")
print("  Now, they use the library's .exchange() function...")

# Alice takes Bob's public key (B) and "mixes" it with her private key (a)
# This computes: S = a * B = a * (b*G)
alice_shared_secret = alice_private_key.exchange(ec.ECDH(), bob_public_key)
print(f"\n  - Alice computes S = a * B:")
print(f"    Alice's Secret ('🟤 BROWN'): {alice_shared_secret.hex()}")
time.sleep(1)

# Bob takes Alice's public key (A) and "mixes" it with his private key (b)
# This computes: S = b * A = b * (a*G)
bob_shared_secret = bob_private_key.exchange(ec.ECDH(), alice_public_key)
print(f"\n  - Bob computes S = b * A:")
print(f"    Bob's Secret ('🟤 BROWN'):   {bob_shared_secret.hex()}\n")
time.sleep(1)

(The Real Exchange)
  Now, they use the library's .exchange() function...

  - Alice computes S = a * B:
    Alice's Secret ('🟤 BROWN'): a5601f52123658f11a0f0d97a6d4bcab34a3ad29aff26dd5dd353ba70d95a0ced7efaff646478453577992e40697ca76

  - Bob computes S = b * A:
    Bob's Secret ('🟤 BROWN'):   a5601f52123658f11a0f0d97a6d4bcab34a3ad29aff26dd5dd353ba70d95a0ced7efaff646478453577992e40697ca76



In [8]:
if alice_shared_secret == bob_shared_secret:
    print("   SUCCESS")
    print("   The two long hex strings are IDENTICAL.")
    print("   This proves the logic works with a real, secure library.")
    print("   They can now use this shared secret for AES encryption.")
else:
    print("   >>> FAILURE! <<<")
    print("   The secrets do not match.")

print("\n\nEnd of Real Crypto Simulation")

   SUCCESS
   The two long hex strings are IDENTICAL.
   This proves the logic works with a real, secure library.
   They can now use this shared secret for AES encryption.


End of Real Crypto Simulation
